In [1]:
import numpy as np

In [2]:
train_images=np.load('data/train-images.npy')

In [3]:
train_images=train_images.reshape((-1,28*28))

In [4]:
train_labels=np.load('data/train-labels.npy')

In [5]:
def sigmoid(x):
    return 1/(np.exp(-x)+1)

In [6]:
def d_sigmoid(x):
    return (np.exp(-x))/((np.exp(-x)+1)**2)

In [7]:
def softmax(x):
    exp_element=np.exp(x-x.max())
    return exp_element/np.sum(exp_element,axis=0)

In [8]:
def d_softmax(x):
    exp_element=np.exp(x-x.max())
    return exp_element/np.sum(exp_element,axis=0)*(1-exp_element/np.sum(exp_element,axis=0))

In [9]:
def init(x,y):
    return np.random.uniform(-1.,1.,(x,y))/np.sqrt(x*y)

In [10]:
np.random.seed(17)

In [11]:
l1=init(28*28,128)

In [12]:
l2=init(128,10)

In [13]:
import apple

In [67]:
ssoftmax=apple.jit('''
λxs.
  { m ⟜ (⋉)/* _1 xs; a ⟜ [e:(x-m)]`{0} xs
  ; sum ← [(+)/* 0 x]
  ; n ⟜ sum a
  ; (%n)`{0} a
  }
''')

In [60]:
vsigmoid=apple.jit("([1%(1+ℯ(_x))]`{0})")

In [16]:
vize=apple.jit("λn.[?x=n,.1::float,.0]'irange 0 9 1")

In [36]:
vmap=apple.jit("((λn.[?x=n,.1::float,.0]'irange 0 9 1)')")

In [17]:
dp=apple.jit("[(+)/ ((*)`((x::Vec n float)) y)]")

In [47]:
mmul=apple.jit("[x%.(y::M float)]")

In [68]:
def fw_bw(x,y):
    # targets = np.zeros((len(y),10), np.float64)
    # targets[range(targets.shape[0]),y] = 1
    targets=apple.f(vmap,y)

    # x_l1p=apple.f(mmul,x,l1)
    x_l1p=x.dot(l1)
    x_sigmoid=softmax(x_l1p)
    # x_sigmoid=apple.f(ssoftmax,x_l1p)
    print(x_l1p.shape,x_sigmoid.shape)
    x_l2p=x_sigmoid.dot(l2)
    out=softmax(x_l2p)

    error=2*(out-targets)/out.shape[0]*d_softmax(x_l2p)
    update_l2=x_sigmoid.T@error


    error=((l2).dot(error.T)).T*d_sigmoid(x_l1p)
    update_l1=x.T@error

    return out,update_l1,update_l2

In [69]:
fw_bw(train_images,train_labels)

(60000, 128) (60000, 128)


(array([[1.66666675e-05, 1.66666668e-05, 1.66666669e-05, ...,
         1.66666668e-05, 1.66666663e-05, 1.66666670e-05],
        [1.66666675e-05, 1.66666669e-05, 1.66666664e-05, ...,
         1.66666661e-05, 1.66666657e-05, 1.66666677e-05],
        [1.66666652e-05, 1.66666672e-05, 1.66666671e-05, ...,
         1.66666673e-05, 1.66666665e-05, 1.66666672e-05],
        ...,
        [1.66666666e-05, 1.66666665e-05, 1.66666667e-05, ...,
         1.66666682e-05, 1.66666661e-05, 1.66666662e-05],
        [1.66666671e-05, 1.66666666e-05, 1.66666675e-05, ...,
         1.66666662e-05, 1.66666669e-05, 1.66666670e-05],
        [1.66666667e-05, 1.66666671e-05, 1.66666667e-05, ...,
         1.66666655e-05, 1.66666664e-05, 1.66666655e-05]]),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([[9.25909